In [1]:
import pandas as pd
import pyarrow.parquet as pq

In [9]:
df_PlayTimeGenre = pd.read_csv("funciones/PlayTimeGenre.csv", low_memory=False)

In [24]:
def PlayTimeGenre(genre: str):
    
    genero_elegido = df_PlayTimeGenre[df_PlayTimeGenre['genre'].str.contains(genre, case=False)]

    if genero_elegido.empty:
        return {"No hay información para el género": genre}
    
    anio = genero_elegido['release_date'].values[0]

    return {"El año con más horas jugadas para el género " + genre + " es el " + str(anio)}

In [29]:
PlayTimeGenre("adventure")

{'El año con más horas jugadas para el género adventure es el 2011'}

In [2]:
df_UserForGenre_1 = pd.read_csv("funciones/UserForGenre_1.csv", low_memory=False)
df_UserForGenre_2 = pd.read_csv("funciones/UserForGenre_2.csv", low_memory=False)

In [34]:
def UserForGenre(genre: str = None):

    if genre == "":
        return {"Si no sabe que buscar, no busque nada."}
    
    genero_elegido = df_UserForGenre_1[df_UserForGenre_1['genre'].str.contains(genre, case=False)]

    if genero_elegido.empty:
        return {"No hay información para el género": genre}

    usuario = genero_elegido['user_id'].values[0]

    usuario_top = df_UserForGenre_2[(df_UserForGenre_2['user_id'] == usuario) & (df_UserForGenre_2['genre'].str.contains(genre, case=False))]

    lista_horas_jugadas = [
    {"Año": anio, "minutos": tiempo} for anio, tiempo in zip(usuario_top['posted'], usuario_top['playtime_forever'])
    ]

    return {
        "Usuario": "El usuario con más horas jugadas para el género " + genre + " es " + usuario,
        "Horas jugadas": lista_horas_jugadas
    }

In [57]:
UserForGenre("asa")

{'No hay información para el género': 'asa'}

In [37]:
df_UsersRecommend = pd.read_csv("funciones/UsersRecommend.csv", low_memory=False)

In [50]:
def UsersRecommend(posted: int):

    if type(posted) != int:
        return ("Ingrese un número.")
    
    df_anio = df_UsersRecommend[df_UsersRecommend['posted'] == posted]

    if df_anio.empty:
        return ("No hay datos para el año indicado.")

    top_3_dict = {}
    for i, row in df_anio.iterrows():
        puesto = "Puesto " + str(i + 1 - df_anio.index[0])
        top_3_dict[puesto] = row['app_name']

    return top_3_dict

In [58]:
UsersRecommend("ca3")

'Ingrese un número.'

In [59]:
df_UsersNotRecommend = pd.read_csv("funciones/UsersNotRecommend.csv", low_memory=False)

In [60]:
def UsersNotRecommend(posted: int):

    if type(posted) != int:
        return ("Ingrese un número.")
    
    df_anio = df_UsersNotRecommend[df_UsersNotRecommend['posted'] == posted]

    if df_anio.empty:
        return ("No hay datos para el año indicado.")
    
    top_3_dict = {}
    for i, row in df_anio.iterrows():
        puesto = "Puesto " + str(i + 1 - df_anio.index[0])
        top_3_dict[puesto] = row['app_name']

    return top_3_dict

In [64]:
UsersNotRecommend(433)

'No hay datos para el año indicado.'

In [65]:
df_sentiment_analysis = pd.read_csv("funciones/sentiment_analysis.csv", low_memory=False)

In [71]:
def Sentiment_Analysis(year: int):

    if type(year) != int:
        return ("Ingrese un número.")
    
    # Filtrar el DataFrame para obtener las filas que coinciden con el release_date dado
    df_anio = df_sentiment_analysis[df_sentiment_analysis['release_date'] == year]
 
    if df_anio.empty:
        return ("No hay datos para el año indicado.")
       
    # Convertir las filas coincidentes a una lista de diccionarios
    filas_lista = df_anio.to_dict(orient='records')
    
    return filas_lista

In [74]:
Sentiment_Analysis(2013)

[{'release_date': 2013, 'Negativo': 1637, 'Neutro': 1555, 'Positivo': 4394}]

In [88]:
df_matriz_similitudes = pd.read_parquet("ML/df_matriz_similitudes.parquet")

In [96]:
def recomendacion_juego(item_id: int):
    N = 5

    if type(item_id) != int:
        return ("Ingrese un id de juego.")  

    if item_id not in df_matriz_similitudes.index:
        return ("No se encuentra un juego con el id ingresado.")

    # Busca la fila de similitud para el juego de referencia
    fila_similitud = df_matriz_similitudes.loc[item_id]

    # Ordena los valores de similitud en orden descendente
    juegos_similares = fila_similitud.sort_values(ascending=False)

    juego_seleccionado = juegos_similares[0:1]
    juegos_recomendados = juegos_similares[1:N+1]  # Excluye el juego de referencia
    

    return {
        "El juego ":juego_seleccionado.index[0], 
        "es similar a estos 5 juegos": juegos_recomendados.index.to_list()       
    }

In [99]:
recomendacion_juego(20)

{'El juego ': 'Team Fortress Classic',
 'es similar a estos 5 juegos': ['Half-Life: Blue Shift',
  'Day of Defeat',
  'DOOM 3',
  'Bloody Good Time',
  'Aliens versus Predator Classic 2000']}

In [14]:

fila_similitud = df_matriz_similitudes.iloc[10]
fila_similitud

Counter-Strike               0.908000
Team Fortress Classic        0.139628
Day of Defeat                0.033290
Deathmatch Classic          -0.116239
Half-Life: Opposing Force    0.365614
                               ...   
True or False               -0.167369
Super Switch                -0.218499
You Have 10 Seconds 2       -0.169409
Galactic Storm              -0.116635
What The Box?               -0.103563
Name: 10, Length: 2834, dtype: float64

In [16]:
juegos_similares = fila_similitud.sort_values(ascending=False)
juegos_similares

Counter-Strike: Source       1.000000
Portal 2                     0.997696
PAYDAY™ The Heist            0.997655
Half-Life 2                  0.995833
Saints Row IV                0.994714
                               ...   
One Day For Ched            -0.217504
Puppy Dog: Jigsaw Puzzles   -0.217601
Real Warfare 1242           -0.217940
Super Switch                -0.218499
Nanofights                  -0.219060
Name: 10, Length: 2834, dtype: float64

In [30]:
juego_seleccionado = juegos_similares[:1]
juego_seleccionado.index[0]

'Counter-Strike: Source'

In [48]:
juegos_recomendados = juegos_similares[1:5+1]
# Formatear la salida
for nombre in juegos_recomendados.index:
    print(nombre)


Portal 2
PAYDAY™ The Heist
Half-Life 2
Saints Row IV
Left 4 Dead 2


In [56]:
#juegos_recomendados.index
juegos_recomendados.index.to_list()

['Portal 2',
 'PAYDAY™ The Heist',
 'Half-Life 2',
 'Saints Row IV',
 'Left 4 Dead 2']